### Imports and macros

In [13]:
from __future__ import annotations
from typing import Callable
from copy import copy, deepcopy
from collections import defaultdict
from functools import reduce
import random
import os
import sys

import numpy as np
import utils
import check_constraints

INTERVALE = 'Intervale'
ZILE = 'Zile'
MATERII = 'Materii'
PROFESORI = 'Profesori'
SALI = 'Sali'
STUDENTI = 'Nr. studenti'
CONSTRANGERI = 'Constrangeri'
PROF_WEEK_COUNT = 'Aparitii profesori'
TEACHER_ORDER = 'Ordine profesori'
PAUSES = 2
ARGUMENTS = 1

#------------------------------------------------------
# MCTS constants

N = 'N'
Q = 'Q'
STATE = 'state'
PARENT = 'parent'
ACTIONS = 'actions'
from math import sqrt, log

# lab utils
# constanta care reglează raportul între explorare și exploatare (CP = 0 -> doar exploatare)
CP = 1.0 / sqrt(2.0)

### Make data more usable for me

In [14]:
class Conditions:
    ''' Parses the data from a file and builds a general dictionary for the algorithms to use '''
    
    def __init__(self, filename=f'inputs/dummy.yaml'):
        self.filename = filename
        self.dict = {}
    
    # keep only valid entries from "Constrangeri"
    def filter_constraints(self, c_list):
        days = []
        intervals = []
        pause = None

        for item in c_list:
            if not item.startswith('!'):
                if '-' in item:
                    # split intervals and convert to tuples
                    start, end = map(int, item.split('-'))
                    intervals.extend([(i, i+2) for i in range(start, end, 2)])
                else:
                    # add the day to the list
                    days.append(item)
            elif 'Pauza' in item:
                pause = int(item.split(' ')[2])
        
        # pause is None if no such constraint exists
        return days, intervals, pause
    
    def parse_data(self):
        ''' Arrange the data as needed '''
        timetable_specs = utils.read_yaml_file(self.filename)
        self.dict[ZILE] = timetable_specs[ZILE]  
        day_occurrences = {day: 0 for day in self.dict[ZILE]}
        
        # generate tuples from comma separated strings
        self.dict[INTERVALE] = list(map(lambda x: tuple(map(int, x.strip('()').split(', '))), timetable_specs[INTERVALE]))
        interval_occurrences = {interval: 0 for interval in self.dict[INTERVALE]}
        
        # initialize each subject's details
        self.dict[MATERII] = {}
        
        for materie in timetable_specs[MATERII]:
            self.dict[MATERII][materie] = {}
            self.dict[MATERII][materie][STUDENTI] = timetable_specs[MATERII][materie]
            self.dict[MATERII][materie][PROFESORI] = []
            self.dict[MATERII][materie][SALI] = []
        
        self.dict[PROFESORI] = timetable_specs[PROFESORI]
        for prof, val in self.dict[PROFESORI].items():
            for materie in val[MATERII]:
                self.dict[MATERII][materie][PROFESORI].append(prof)
            
            # also count dta for the frequence of the intervals
            self.dict[PROFESORI][prof][CONSTRANGERI] = self.filter_constraints(self.dict[PROFESORI][prof][CONSTRANGERI])
            for day in self.dict[PROFESORI][prof][CONSTRANGERI][0]:
                day_occurrences[day] += 1
            for interval in self.dict[PROFESORI][prof][CONSTRANGERI][1]:
                interval_occurrences[interval] += 1
        
        # sort the days and intervals based on preference
        day_occurrences = sorted(day_occurrences.items(), key=lambda x: x[1])
        self.dict[ZILE] = [day for day, _ in day_occurrences]
        
        interval_occurrences = sorted(interval_occurrences.items(), key=lambda x: x[1])
        self.dict[INTERVALE] = [interval for interval, _ in interval_occurrences]
            
        # for each subject order the teachers based on the number of intervals they can teach in
        for materie in self.dict[MATERII]:
            self.dict[MATERII][materie][PROFESORI] = sorted(self.dict[MATERII][materie][PROFESORI],
                key=lambda prof: len(self.dict[PROFESORI][prof][CONSTRANGERI][0]) * len(self.dict[PROFESORI][prof][CONSTRANGERI][1]))
                
        self.dict[SALI] = timetable_specs[SALI]
        for sala, val in self.dict[SALI].items():
            for materie in val[MATERII]:
                self.dict[MATERII][materie][SALI].append(sala)
        
        # sort the rooms in ascending order
        for materie in self.dict[MATERII]:
            self.dict[MATERII][materie][SALI] = sorted(self.dict[MATERII][materie][SALI],
                key=lambda room: self.dict[SALI][room]['Capacitate'])
        
                
        # order all the teachers based on the number of intervals they can teach in
        self.dict[TEACHER_ORDER] = []
        for prof in self.dict[PROFESORI].keys():
            self.dict[TEACHER_ORDER].append(prof)
        self.dict[TEACHER_ORDER] = sorted(self.dict[TEACHER_ORDER],
            key=lambda prof: len(self.dict[PROFESORI][prof][CONSTRANGERI][0]) * len(self.dict[PROFESORI][prof][CONSTRANGERI][1]))
            
cond = Conditions(f'inputs/orar_bonus_exact.yaml')
cond.parse_data()
print(cond.dict)

{'Zile': ['Vineri', 'Luni', 'Marti', 'Joi', 'Miercuri'], 'Intervale': [(16, 18), (18, 20), (12, 14), (14, 16), (10, 12), (8, 10)], 'Materii': {'DS': {'Nr. studenti': 515, 'Profesori': ['Sergiu Adamescu', 'Elena Stoica', 'Mihai Moldovan', 'Ruxandra Scarlatescu', 'Roxana Epure', 'Cristian Adamescu', 'Elena Andronescu'], 'Sali': ['ED009', 'ED063', 'EG363', 'EC138']}, 'MS': {'Nr. studenti': 545, 'Profesori': ['Pavel Georgescu', 'Cristian Moldovan', 'Mihai Dumitrescu', 'Vlad Epure', 'Pavel Vasilescu', 'Cristian Adamescu', 'Denisa Popa', 'Roxana Popescu', 'Alexandru Vasilescu', 'Ruxandra Chiriac', 'Alexandru Ilie'], 'Sali': ['ED009', 'ED063', 'EG363', 'EC138']}, 'PCom': {'Nr. studenti': 510, 'Profesori': ['Mihai Dumitrescu', 'Vlad Epure', 'Pavel Vasilescu', 'Roxana Epure', 'Andrei Ionescu', 'Denisa Popa', 'Elena Andronescu', 'Ruxandra Chiriac'], 'Sali': ['ED027', 'ED063', 'EG363']}, 'PM': {'Nr. studenti': 500, 'Profesori': ['Petru Chiriac', 'Ruxandra Georgescu', 'Andrei Ionescu', 'Ruxandra C

### State definition

In [15]:
class State:
    ''' Encapsulation of the board and other utils '''
    def __init__(
        self,
        timetable_specs,
        board,
        conflicts,
        cost,
        unassigned,
        teacher_week_count,
        filename=f'inputs/orar_mic_exact.yaml',
        seed = 42,
    ) -> None:
        self.timetable_specs = timetable_specs
        self.filename = filename
        self.board = board if board is not None else self.generate_board()
        
        # soft conflicts counter
        self.nconflicts = conflicts if conflicts is not None else self.__compute_initial_conflicts()
        
        # cost for unassigned students basically
        self.cost = cost if cost is not None else sum([self.timetable_specs[MATERII][materie][STUDENTI] for materie in self.timetable_specs[MATERII]])
        self.unassigned = unassigned if unassigned is not None else self.get_ordered_subjects_rooms_teachers_students()
        self.teacher_week_count = teacher_week_count if teacher_week_count is not None else self.build_week_count()
    
    def generate_board(self):
        ''' Generate an empty, but initialized board '''
        board = {}
    
        for day in self.timetable_specs[ZILE]:
            board[day] = {}
            for interval in self.timetable_specs[INTERVALE]:
                board[day][interval] = {}
                for room in self.timetable_specs[SALI]:
                    board[day][interval][room] = None
        return board
    
    def get_ordered_subjects_rooms_teachers_students(self):
        ''' Order the subjects based on a succession of criteria '''
        tuples = [(materie, self.timetable_specs[MATERII][materie][STUDENTI]) for materie in self.timetable_specs[MATERII]]
        return sorted(tuples, key=lambda x: (len(self.timetable_specs[MATERII][x[0]][SALI]),
                                             len(self.timetable_specs[MATERII][x[0]][PROFESORI]),
                                             self.timetable_specs[MATERII][x[0]][STUDENTI]
                                             ))
    
    def build_week_count(self):
        ''' Initialize how many intervals a teacher has '''
        ret = {}
        for prof, _ in self.timetable_specs[PROFESORI].items():
            ret[prof] = 0
        return ret
    
    def __compute_initial_conflicts(self):
        ''' Initial Soft conflicts '''
        return 0
    
    def is_final(self) -> bool:
        ''' is final if we have all students assigned '''
        for (_, no_stud) in self.unassigned:
            if no_stud > 0:
                return False
        return True
    
    def display(self, output_dir = 'outputs_hill'):
        ''' Generate a _board that is in the suitable printing format and write into a new file '''
        
        _board = deepcopy(self.board)
        day_values = {'Luni':1, 'Marti':2, 'Miercuri':3, 'Joi':4, 'Vineri':5}
        _board = {key: value for key, value in sorted(_board.items(), key=lambda x: day_values[x[0]])}
        
        interval_values = {(8, 10):1, (10, 12):2, (12, 14):3, (14, 16):4, (16, 18):5, (18, 20): 6}
        for day in _board:
            _board[day] = {key: value for key, value in sorted(_board[day].items(), key=lambda x: interval_values[x[0]])}
        
        output_filename = self.filename.replace('inputs', output_dir)
        output_filename = output_filename.replace('yaml', 'txt')
        directory = os.path.dirname(output_filename)

        # Create the directory if it doesn't exist
        if not os.path.exists(directory):
            os.makedirs(directory)
        
        # write the table into it
        with open(output_filename, 'w') as file:
            file.write(utils.pretty_print_timetable(_board, self.filename))
        return _board
        
    def clone(self) -> State:
        return State(self.timetable_specs, deepcopy(self.board), self.nconflicts, self.cost, 
                     deepcopy(self.unassigned), deepcopy(self.teacher_week_count), self.filename)
    
    def heuristic_eval(self):
        ''' Comparison formula for the neoghbours '''
        return 2 * self.cost + self.nconflicts
    
    def get_next_states(self):
        ret = []
        for (new_subject, no_stud) in self.unassigned:
            if no_stud > 0:
                for day in self.timetable_specs[ZILE]:
                    for interval in self.timetable_specs[INTERVALE]:
                        for teacher in self.timetable_specs[MATERII][new_subject][PROFESORI]:
                            if self.teacher_week_count[teacher] >= 7:
                                continue
                            for room in self.timetable_specs[MATERII][new_subject][SALI]:
                                ok = 1
                                
                                # check if the teacher already has a slot in that interval
                                for other_room in self.timetable_specs[SALI]:
                                    if room != other_room and self.board[day][interval][other_room] is not None \
                                        and self.board[day][interval][other_room][0] == teacher:
                                            ok = 0
                                            break
                                # the teacher does not teach in that interval in another room
                                if ok == 1:
                                    ret.append(self.fill_occupied_slot(day, interval, room, teacher, new_subject))
        return ret
    
    def check_failed_optional_constraint(self, day, interval, new_teacher):
        ''' count new soft conflicts created by this state '''
        total = 0
        if day not in self.timetable_specs[PROFESORI][new_teacher][CONSTRANGERI][0]:
            total += 5
        if interval not in self.timetable_specs[PROFESORI][new_teacher][CONSTRANGERI][1]:
            total += 2
        
        if self.timetable_specs[PROFESORI][new_teacher][CONSTRANGERI][PAUSES] is not None:
            last_end_time = None
            last_start_time = None
            _start = interval[0]
            _end = interval[1]
            end_times = []
            start_times = []
            
            interval_values = {(8, 10):1, (10, 12):2, (12, 14):3, (14, 16):4, (16, 18):5, (18, 20): 6}
            for other_interval in interval_values.keys():
                for room in self.board[day][other_interval]:
                    if self.board[day][interval][room]:
                        crt_prof, _ = self.board[day][interval][room]
                        if new_teacher == crt_prof:
                            end_times.append(other_interval[1])
                            start_times.append(other_interval[0])
                            
            for end in end_times:
                if end < _start:
                    last_end_time = end
            if last_end_time is not None:
                if _start - last_end_time > self.timetable_specs[PROFESORI][new_teacher][CONSTRANGERI][PAUSES]:
                    total += 2 * (_start - last_end_time - self.timetable_specs[PROFESORI][new_teacher][CONSTRANGERI][PAUSES])
                    
            for start in start_times:
                if start > _end:
                    last_start_time = start
                    break
            if last_start_time is not None:
                if last_start_time - _end > self.timetable_specs[PROFESORI][new_teacher][CONSTRANGERI][PAUSES]:
                    total += 2 * (last_start_time - _end - self.timetable_specs[PROFESORI][new_teacher][CONSTRANGERI][PAUSES])
        return total
    
    # add a group of students from a subject to an empty room
    # check if an optional constraint has been denied (_conflicts + 1)
    # reduce the number of students unassigned
    def fill_empty_slot(self, day, interval, room, new_teacher, subject):
        new_state = deepcopy(self.board)
        _cost = self.cost
        _unassigned = deepcopy(self.unassigned)
        _conflicts = self.nconflicts
        _teacher_wk_count = deepcopy(self.teacher_week_count)
        _teacher_wk_count[new_teacher] += 1
        
        new_state[day][interval][room] = (new_teacher, subject)
        _conflicts += self.check_failed_optional_constraint(day, interval, new_teacher)
                
        for i, (sub, no) in enumerate(_unassigned):
            if sub == subject:
                # we might get with negative students assigned, it is OK
                subtract = self.timetable_specs[SALI][room]['Capacitate']
                _unassigned[i] = (sub, no - subtract)
                _cost -= subtract
                if _unassigned[i][1] < 0:
                    # it's actually an increase in cost
                    _cost -= _unassigned[i][1]
                break
        return State(self.timetable_specs, new_state, _conflicts, _cost, _unassigned, _teacher_wk_count, self.filename)
    
    # add a group of students back to unassigned to make room for new ones
    def fill_occupied_slot(self, day, interval, room, new_teacher, new_subject):
        new_state = deepcopy(self.board)
        if new_state[day][interval][room] is None:
            return self.fill_empty_slot(day, interval, room, new_teacher, new_subject)
        _cost = self.cost
        _unassigned = deepcopy(self.unassigned)
        _conflicts = self.nconflicts
        
        (old_teacher, old_subject) = new_state[day][interval][room]
        new_state[day][interval][room] = (new_teacher, new_subject)
        
        _teacher_wk_count = deepcopy(self.teacher_week_count)
        _teacher_wk_count[new_teacher] += 1
        _teacher_wk_count[old_teacher] -= 1
        
        _conflicts -= self.check_failed_optional_constraint(day, interval, old_teacher)
            
        for i, (sub, no) in enumerate(_unassigned):
            if sub == old_subject:
                add = self.timetable_specs[SALI][room]['Capacitate']
                _cost += add
                _unassigned[i] = (sub, no + add)
            if sub == new_subject:
                subtract = self.timetable_specs[SALI][room]['Capacitate']
                _cost -= subtract
                _unassigned[i] = (sub, no - subtract)
        
        
        return State(self.timetable_specs, new_state, _conflicts, _cost, _unassigned, _teacher_wk_count, self.filename)


### Make a run environment, just like in lab

In [16]:
statistics = {}
def run_test(method: Callable, filename: str, **kwargs) -> None:
    ''' Testeaza un algoritm (method) din fisierul filename '''
    
    # parse the file and generate initial state
    cond = Conditions(filename)
    cond.parse_data()
    initial = State(cond.dict, None, None, None, None, None, filename, seed=random.random())
    
    is_final, iters, states, state = method(initial, **kwargs)
    
    # write output and check conflicts
    _board = state.display()
    print("Number of HARD restrictions failed:")
    print(check_constraints.check_mandatory_constraints(_board, utils.read_yaml_file(state.filename)))
    print("Number of SOFT restrictions failed:")
    print(check_constraints.check_optional_constraints(_board, utils.read_yaml_file(state.filename)))
    
    print(f"Number of iterations: {iters:,}")
    print(f"Number of states generated: {states:,}")
    stat = {
        "final_state": is_final,
        "iter": iters,
        "nums": states
    }
    statistics[filename] = stat

### Basic HC

In [17]:
def hill_climbing(initial: State, max_iters: int = 250):
    ''' hill climbing best first method '''
    iters, states = 0, 0
    state = initial.clone()
    
    # put a threshold for the number of iterations
    while iters < max_iters:
        iters += 1
        next_states = state.get_next_states()
        new_states_no = len(next_states)
        if new_states_no == 0:
            break
        states += new_states_no
        
        # get best next state
        best_state = min(next_states, key=lambda x: x.heuristic_eval())
        if best_state.heuristic_eval() >= state.heuristic_eval():
            break
        
        state = best_state
        
    return state.is_final(), iters, states, state

### Hill Climbing - Dummy

In [18]:
%time run_test(hill_climbing, f'inputs/dummy.yaml')

Number of HARD restrictions failed:
0
Number of SOFT restrictions failed:
0
Number of iterations: 12
Number of states generated: 544
CPU times: total: 62.5 ms
Wall time: 64 ms


### Hill Climbing - Mic

In [19]:
%time run_test(hill_climbing, f'inputs/orar_mic_exact.yaml')

Number of HARD restrictions failed:
0
Number of SOFT restrictions failed:
0
Number of iterations: 34
Number of states generated: 20,535
CPU times: total: 4.86 s
Wall time: 4.86 s


### Hill Climbing - Mediu

In [20]:
%time run_test(hill_climbing, f'inputs/orar_mediu_relaxat.yaml')

Number of HARD restrictions failed:
0
Number of SOFT restrictions failed:
0
Number of iterations: 48
Number of states generated: 146,677
CPU times: total: 48 s
Wall time: 47.9 s


### Hill Climbing - Mare

In [21]:
%time run_test(hill_climbing, f'inputs/orar_mare_relaxat.yaml')

Number of HARD restrictions failed:
0
Number of SOFT restrictions failed:
0
Number of iterations: 52
Number of states generated: 379,176
CPU times: total: 2min 35s
Wall time: 2min 36s


### Hill Climbing - Constrans

In [22]:
%time run_test(hill_climbing, f'inputs/orar_constrans_incalcat.yaml')

Number of HARD restrictions failed:
0
Number of SOFT restrictions failed:
0
Number of iterations: 55
Number of states generated: 79,082
CPU times: total: 19.7 s
Wall time: 19.8 s


### Hill Climbing - Bonus

In [23]:
%time run_test(hill_climbing, f'inputs/orar_bonus_exact.yaml')

Number of HARD restrictions failed:
0
Number of SOFT restrictions failed:
Profesorul Denisa Popa are pauza prea mare de la 10 pana la 18!
Profesorul Mihai Moldovan are pauza prea mare de la 12 pana la 14!
Profesorul Ruxandra Chiriac are pauza prea mare de la 10 pana la 12!
Profesorul Ruxandra Chiriac are pauza prea mare de la 16 pana la 18!
Profesorul Ruxandra Chiriac are pauza prea mare de la 14 pana la 18!
5
Number of iterations: 105
Number of states generated: 325,196
CPU times: total: 2min 4s
Wall time: 2min 4s


# MCTS states and functions

In [71]:
class AlterState:
    def __init__(
        self,
        timetable_specs,
        board,
        conflicts,
        cost,
        unassigned,
        teacher_week_count,
        filename=f'inputs/orar_mic_exact.yaml',
        seed = 42,
    ) -> None:
        self.timetable_specs = timetable_specs
        self.filename = filename
        self.board = board if board is not None else self.generate_board()
        
        # save the output to avoid further computation for next states and the probabilities for them
        self.next_states_list = None
        
        # soft conflicts counter
        self.nconflicts = conflicts if conflicts is not None else self.__compute_initial_conflicts()
        
        # cost for unassigned students basically
        self.cost = cost if cost is not None else sum([self.timetable_specs[MATERII][materie][STUDENTI] for materie in self.timetable_specs[MATERII]])
        self.unassigned = unassigned if unassigned is not None else self.get_ordered_subjects_rooms_teachers_students()
        self.teacher_week_count = teacher_week_count if teacher_week_count is not None else self.build_week_count()
        
    def generate_board(self):
        ''' Generate an empty, but initialized board '''
        board = {}
    
        for day in self.timetable_specs[ZILE]:
            board[day] = {}
            for interval in self.timetable_specs[INTERVALE]:
                board[day][interval] = {}
                for room in self.timetable_specs[SALI]:
                    board[day][interval][room] = None
        return board
    
    def get_ordered_subjects_rooms_teachers_students(self):
        ''' Order the subjects based on a succession of criteria '''
        tuples = [(materie, self.timetable_specs[MATERII][materie][STUDENTI]) for materie in self.timetable_specs[MATERII]]
        return sorted(tuples, key=lambda x: (len(self.timetable_specs[MATERII][x[0]][SALI]),
                                             len(self.timetable_specs[MATERII][x[0]][PROFESORI]),
                                             self.timetable_specs[MATERII][x[0]][STUDENTI]
                                             ))
    
    @staticmethod
    def generate_exponential_distribution(n, ratio=1.5):
        ''' generate probabilities, stating the first elements should occur more '''
        lambdas = np.array([1 / (ratio ** i) for i in range(n)], dtype=float)
    
        # Normalize lambdas to ensure the sum equals 1
        lambdas /= np.sum(lambdas)
        return lambdas
    
    def build_week_count(self):
        ''' Initialize how many intervals a teacher has '''
        ret = {}
        for prof, _ in self.timetable_specs[PROFESORI].items():
            ret[prof] = 0
        return ret
    
    def __compute_initial_conflicts(self):
        """ Initialize soft conflicts with 0"""
        return 0
    
    def heuristic_eval(self):
        ''' compute the potential of the next states '''
        return self.cost + self.nconflicts
    
    def is_final(self) -> bool:
        available_states = self.get_available_states()
        if len(available_states) == 0:
            return True
        for (_, no_stud) in self.unassigned:
            if no_stud > 0:
                return False
        return True
    
    def display(self, output_dir = 'outputs_mcts'):
        ''' write the timetable to a new file'''
        
        _board = deepcopy(self.board)
        day_values = {'Luni':1, 'Marti':2, 'Miercuri':3, 'Joi':4, 'Vineri':5}
        _board = {key: value for key, value in sorted(_board.items(), key=lambda x: day_values[x[0]])}
        
        interval_values = {(8, 10):1, (10, 12):2, (12, 14):3, (14, 16):4, (16, 18):5, (18, 20): 6}
        for day in _board:
            _board[day] = {key: value for key, value in sorted(_board[day].items(), key=lambda x: interval_values[x[0]])}
        
        output_filename = self.filename.replace('inputs', output_dir)
        output_filename = output_filename.replace('yaml', 'txt')
        directory = os.path.dirname(output_filename)

        # Create the directory if it doesn't exist
        if not os.path.exists(directory):
            os.makedirs(directory)
        
        # write the table into it
        with open(output_filename, 'w') as file:
            file.write(utils.pretty_print_timetable(_board, self.filename))
        return _board
    
    # add a group of students from a subject to an empty room
    # check if an optional constraint has been denied (_conflicts + 1)
    # reduce the number of students unassigned
    def fill_empty_slot(self, day, interval, room, new_teacher, subject):
        new_state = deepcopy(self.board)
        _cost = self.cost
        _unassigned = deepcopy(self.unassigned)
        _conflicts = self.nconflicts
        _teacher_wk_count = deepcopy(self.teacher_week_count)
        _teacher_wk_count[new_teacher] += 1
        
        new_state[day][interval][room] = (new_teacher, subject)
        _conflicts += self.check_failed_optional_constraint(day, interval, new_teacher)
                
        for i, (sub, no) in enumerate(_unassigned):
            if sub == subject:
                # we might get with negative students assigned, it is OK
                subtract = self.timetable_specs[SALI][room]['Capacitate']
                _cost -= subtract
                _unassigned[i] = (sub, no - subtract)
                if _unassigned[i][1] < 0:
                    # it's actually an increase in cost
                    _cost -= _unassigned[i][1]
                break
        return AlterState(self.timetable_specs, new_state, _conflicts, _cost, _unassigned, _teacher_wk_count, self.filename)
    
    def compute_possible_cost(self, day, interval, room, new_teacher, subject):
        _cost = self.cost
        _conflicts = self.nconflicts
        _conflicts += self.check_failed_optional_constraint(day, interval, new_teacher)
        for _, (sub, no) in enumerate(self.unassigned):
            if sub == subject:
                # we might get with negative students assigned, it is OK
                subtract = self.timetable_specs[SALI][room]['Capacitate']
                _cost -= subtract
                if (no - subtract) < 0:
                    _cost -= (no - subtract)
                break
        return _cost + _conflicts
    
    def check_failed_optional_constraint(self, day, interval, new_teacher):
        ''' Compute how many optional constraints the next move fails '''
        
        total = 0
        if day not in self.timetable_specs[PROFESORI][new_teacher][CONSTRANGERI][0]:
            total += 5
        if interval not in self.timetable_specs[PROFESORI][new_teacher][CONSTRANGERI][1]:
            total += 2
            
        if self.timetable_specs[PROFESORI][new_teacher][CONSTRANGERI][PAUSES] is not None:
            last_end_time = None
            last_start_time = None
            _start = interval[0]
            _end = interval[1]
            end_times = []
            start_times = []
            
            interval_values = {(8, 10):1, (10, 12):2, (12, 14):3, (14, 16):4, (16, 18):5, (18, 20): 6}
            for other_interval in interval_values.keys():
                for room in self.board[day][other_interval]:
                    if self.board[day][interval][room]:
                        crt_prof, _ = self.board[day][interval][room]
                        if new_teacher == crt_prof:
                            end_times.append(other_interval[1])
                            start_times.append(other_interval[0])
                            
            for end in end_times:
                if end < _start:
                    last_end_time = end
            if last_end_time is not None:
                if _start - last_end_time > self.timetable_specs[PROFESORI][new_teacher][CONSTRANGERI][PAUSES]:
                    total += 2 * (_start - last_end_time - self.timetable_specs[PROFESORI][new_teacher][CONSTRANGERI][PAUSES])
                    
            for start in start_times:
                if start > _end:
                    last_start_time = start
                    break
            if last_start_time is not None:
                if last_start_time - _end > self.timetable_specs[PROFESORI][new_teacher][CONSTRANGERI][PAUSES]:
                    total += 2 * (last_start_time - _end - self.timetable_specs[PROFESORI][new_teacher][CONSTRANGERI][PAUSES])
        
        return total
    
    def get_available_states(self):
        ''' get the next neighbours '''
        if self.next_states_list is not None:
            return self.next_states_list
        ret = []
        for (new_subject, no_stud) in self.unassigned:
            if no_stud > 0:
                for day in self.timetable_specs[ZILE]:
                    for interval in self.timetable_specs[INTERVALE]:
                        for teacher in self.timetable_specs[MATERII][new_subject][PROFESORI]:
                            if self.teacher_week_count[teacher] >= 7:
                                continue
                            for room in self.timetable_specs[MATERII][new_subject][SALI]:
                                # check if the slot is empty
                                if self.board[day][interval][room] is not None:
                                    continue
                                
                                ok = 1  
                                # check if the teacher already has a slot in that interval
                                for other_room in self.timetable_specs[SALI]:
                                    if room != other_room and self.board[day][interval][other_room] is not None \
                                        and self.board[day][interval][other_room][0] == teacher:
                                            ok = 0
                                            break
                                # the teacher does not teach in that interval in another room
                                if ok == 1:
                                    ret.append((self.compute_possible_cost(day, interval, room, teacher, new_subject), (day, interval, room, teacher, new_subject)))
                                    
        ret = sorted(ret, key=lambda x: x[0])[:min(len(ret), 15)]
        self.next_states_list = ret
        return ret
    
    def other_display(self, output_dir = 'custom_outputs'):
        ''' output to a custom file '''
        _board = deepcopy(self.board)
        day_values = {'Luni':1, 'Marti':2, 'Miercuri':3, 'Joi':4, 'Vineri':5}
        _board = {key: value for key, value in sorted(_board.items(), key=lambda x: day_values[x[0]])}
        
        interval_values = {(8, 10):1, (10, 12):2, (12, 14):3, (14, 16):4, (16, 18):5, (18, 20): 6}
        for day in _board:
            _board[day] = {key: value for key, value in sorted(_board[day].items(), key=lambda x: interval_values[x[0]])}
        
        output_filename = self.filename.replace('inputs', output_dir)
        output_filename = output_filename.replace('yaml', 'txt')
        directory = output_dir

        # Create the directory if it doesn't exist
        if not os.path.exists(directory):
            os.makedirs(directory)
        
        # write the table into it
        with open(output_filename, 'a') as file:
            file.write(utils.pretty_print_timetable(_board, self.filename))
            file.write("Number of HARD restrictions failed:\n")
            file.write(str(check_constraints.check_mandatory_constraints(_board, check_constraints.read_yaml_file(self.filename))) + '\n')
            file.write("Number of SOFT restrictions failed:\n")
            file.write(str(check_constraints.check_optional_constraints(_board, check_constraints.read_yaml_file(self.filename))) + '\n')
    


### Just some basic run to test the functionality of the methods

In [72]:
cond = Conditions(f'inputs/orar_mic_exact.yaml')
cond.parse_data()
state = AlterState(cond.dict, None, None, None, None, None, f'inputs/orar_mic_exact.yaml', seed=random.random())
state_dict = {}
while not state.is_final():
    possible_states = state.get_available_states()
    if len(possible_states) == 0:
        break
    state = state.fill_empty_slot(*(random.choice(possible_states)[ARGUMENTS]))
    
_board = state.display('outputs_mcts')

### Initial node and selection criteria

In [73]:
def init_node(state, parent = None):
    return {N: 0, Q: 0, STATE: state, PARENT: parent, ACTIONS: {}}

def select_action_index(node, c = CP):
    N_node = node[N]
    scores_list = list(map(lambda entry: (entry[0], entry[1][Q] / entry[1][N] + c * sqrt(2 * log(N_node) / entry[1][N])), node[ACTIONS].items()))
    return reduce(lambda a, b: a if a[1] > b[1] else b, scores_list)[0]

def next_subject_to_complete(state):
    for i, (_, no_stud) in enumerate(state.unassigned):
        if no_stud > 0:
            return i

probabilities = AlterState.generate_exponential_distribution(n=15, ratio=2)
action = np.random.choice(np.arange(15), size=20, p=probabilities)
print(action)

[1 0 0 3 1 1 0 0 0 0 4 0 0 3 0 3 0 1 2 0]


### MCTS per se

In [74]:
def mcts(state0, budget, tree, ratio=1.5):
    
    states_considered = 0
    states_generated = 0
    
    # initialize tree if necessary
    if tree is None:
        tree = init_node(state0)
    
    for _ in range(budget):
        node = tree
        
        # list of tuples (cost, tuple of arguments for the buildup of the next state)
        actions_list = None
        
        # selection
        while True:
            if node[STATE].is_final():
                break
            
            # list of tuples (cost, tuple of arguments for the buildup of the next state)
            actions_list = node[STATE].get_available_states()
            if len(list(filter(lambda x: actions_list.index(x) in node[ACTIONS].keys(), actions_list))) != len(actions_list):
                break
            
            action_index = select_action_index(node)
            node = node[ACTIONS][action_index]
        
        # expansion
        if not node[STATE].is_final():
            list_to_choose_from = list(filter(lambda x: x not in node[ACTIONS].keys(), range(len(actions_list))))
            n = len(list_to_choose_from)
            probabilities = AlterState.generate_exponential_distribution(n=n, ratio=ratio)
            action = np.random.choice(np.arange(n), size=1, p=probabilities)[0]
            node[ACTIONS][action] = init_node(node[STATE].fill_empty_slot(*(actions_list[action][ARGUMENTS])), parent=node)
            node = node[ACTIONS][action]
            
        # simulation
        state = node[STATE]
        
        while (not state.is_final()):
            list_to_choose_from = state.get_available_states()
            n = len(list_to_choose_from)
            probabilities = AlterState.generate_exponential_distribution(n=n, ratio=ratio)
            action = np.random.choice(np.arange(n), size=1, p=probabilities)[0]
            state = state.fill_empty_slot(*(list_to_choose_from[action][ARGUMENTS]))
            states_considered += n
            states_generated += 1
            
        # reward
        if state.nconflicts != 0:
            reward = state.nconflicts * (-10)
        else:
            reward = 300
            
        # reward propagation
        while node:
            node[N] = node[N] + 1
            node[Q] = node[Q] + reward
            node = node[PARENT]
    
    # choose the best action/state
    # tree is not None
    final_action = select_action_index(tree, 0.0)
    return states_considered, states_generated, tree[ACTIONS][final_action]

### More tests

In [75]:
cond = Conditions(f'inputs/dummy.yaml')
cond.parse_data()
state0 = AlterState(cond.dict, None, None, None, None, None, f'inputs/dummy.yaml', seed=random.random())
state_dict = {}

states_considered, states_generated, tree = mcts(state0, 1, None, ratio=1.5)
print(tree)

{'N': 1, 'Q': -40, 'state': <__main__.AlterState object at 0x000001BF02B646D0>, 'parent': {'N': 1, 'Q': -40, 'state': <__main__.AlterState object at 0x000001BF02B65540>, 'parent': None, 'actions': {0: {...}}}, 'actions': {}}


### Running environment MCTS

In [76]:
def run_mcts(filename, budget=10, ratio=1.5):
    cond = Conditions(filename)
    cond.parse_data()
    state = AlterState(cond.dict, None, None, None, None, None, filename, seed=random.random())
    
    tree = None
    iters = 0
    states_considered = 0
    states_generated = 0
    while state and not state.is_final():
        iters += 1
        s1, s2, tree = mcts(state, budget, tree, ratio)
        state = tree[STATE]
        states_considered += s1
        states_generated += s2
    
    _board = state.display('outputs_mcts')
    print("Number of HARD restrictions failed:")
    print(check_constraints.check_mandatory_constraints(_board, utils.read_yaml_file(state.filename)))
    print("Number of SOFT restrictions failed:")
    print(check_constraints.check_optional_constraints(_board, utils.read_yaml_file(state.filename)))
    print(f"Number of iterations: {iters:,}")
    print(f"Number of states taken into consideration as simulation options: {states_considered:,}")
    print(f"Number of states actually generated: {states_generated:,}")

### MCTS - Dummy

In [50]:
%time run_mcts(f'inputs/dummy.yaml', budget=60, ratio=2.3)

Number of HARD restrictions failed:
0
Number of SOFT restrictions failed:
0
Number of iterations: 11
Number of states taken into consideration as simulation options: 49,481
Number of states actually generated: 3,300
CPU times: total: 625 ms
Wall time: 622 ms


### MCTS - Mic

In [51]:
%time run_mcts(f'inputs/orar_mic_exact.yaml', budget=20, ratio=2)

Number of HARD restrictions failed:
0
Number of SOFT restrictions failed:
0
Number of iterations: 33
Number of states taken into consideration as simulation options: 158,400
Number of states actually generated: 10,560
CPU times: total: 7.16 s
Wall time: 7.15 s


### MCTS - Mediu

In [52]:
%time run_mcts(f'inputs/orar_mediu_relaxat.yaml', budget=20, ratio=2)

Number of HARD restrictions failed:
0
Number of SOFT restrictions failed:
0
Number of iterations: 47
Number of states taken into consideration as simulation options: 324,300
Number of states actually generated: 21,620
CPU times: total: 1min 36s
Wall time: 1min 36s


### MCTS - Mare

In [53]:
%time run_mcts(f'inputs/orar_mare_relaxat.yaml', budget=20, ratio=1.5)

Number of HARD restrictions failed:
0
Number of SOFT restrictions failed:
0
Number of iterations: 51
Number of states taken into consideration as simulation options: 382,500
Number of states actually generated: 25,500
CPU times: total: 5min 16s
Wall time: 5min 17s


### MCTS - Constrans

In [57]:
%time run_mcts(f'inputs/orar_constrans_incalcat.yaml', budget=50, ratio=2)

Number of HARD restrictions failed:
0
Number of SOFT restrictions failed:
0
Number of iterations: 54
Number of states taken into consideration as simulation options: 1,073,280
Number of states actually generated: 71,552
CPU times: total: 1min 21s
Wall time: 1min 21s


### MCTS - Bonus

In [58]:
%time run_mcts(f'inputs/orar_bonus_exact.yaml', budget=10, ratio=2.3)

Number of HARD restrictions failed:
0
Number of SOFT restrictions failed:
Profesorul Cristian Adamescu are pauza prea mare de la 10 pana la 16!
Profesorul Roxana Popescu are pauza prea mare de la 10 pana la 14!
Profesorul Roxana Popescu are pauza prea mare de la 10 pana la 14!
3
Number of iterations: 104
Number of states taken into consideration as simulation options: 803,370
Number of states actually generated: 53,558
CPU times: total: 6min 47s
Wall time: 6min 47s


### MCTS - Bonus - mai bine decat hill
##### Rulat inaintea implementarii calculului de iteratii, bagand in seama doar urmatoarea materie necompletata

In [71]:
%time run_mcts(f'inputs/orar_bonus_exact.yaml', budget=10, ratio=2.3)

Number of HARD restrictions failed:
0
Number of SOFT restrictions failed:
Profesorul Alexandru Ilie are pauza prea mare de la 14 pana la 16!
Profesorul Roxana Epure are pauza prea mare de la 14 pana la 18!
Profesorul Ruxandra Chiriac are pauza prea mare de la 12 pana la 16!
3
CPU times: total: 2min 4s
Wall time: 2min 5s


# Multiple MCTS custom runs

In [89]:
def run_custom(filename, budget=10, ratio=1.5, runs=10):
    cond = Conditions(filename)
    cond.parse_data()
    
    for i in range(runs):
        print('--------------------------------------------')
        print(f'Iter no. {i + 1}:')
        
        state = AlterState(cond.dict, None, None, None, None, None, filename, seed=random.random())
        
        tree = None
        iters = 0
        states_considered = 0
        states_generated = 0
        while state and not state.is_final():
            iters += 1
            s1, s2, tree = mcts(state, budget, tree, ratio)
            state = tree[STATE]
            states_considered += s1
            states_generated += s2
        
        _board = state.other_display()
        print(f"Number of iterations: {iters:,}")
        print(f"Number of states taken into consideration as simulation options: {states_considered:,}")
        print(f"Number of states actually generated: {states_generated:,}")

##### Total time for 10 runs will be displayed (and the errors, since they are printed and not added to file)

In [93]:
%time run_custom(f'inputs/dummy.yaml', budget=50, ratio=2, runs=10)

--------------------------------------------
Iter no. 1:
Number of iterations: 11
Number of states taken into consideration as simulation options: 41,167
Number of states actually generated: 2,750
--------------------------------------------
Iter no. 2:
Number of iterations: 11
Number of states taken into consideration as simulation options: 40,244
Number of states actually generated: 2,750
--------------------------------------------
Iter no. 3:
Number of iterations: 11
Number of states taken into consideration as simulation options: 39,728
Number of states actually generated: 2,750
--------------------------------------------
Iter no. 4:
Number of iterations: 11
Number of states taken into consideration as simulation options: 40,980
Number of states actually generated: 2,750
--------------------------------------------
Iter no. 5:
Number of iterations: 11
Number of states taken into consideration as simulation options: 39,782
Number of states actually generated: 2,750
---------------

In [88]:
%time run_custom(f'inputs/orar_constrans_incalcat.yaml', budget=40, ratio=2, runs=10)

--------------------------------------------
Iter no. 1:
--------------------------------------------
Iter no. 2:
--------------------------------------------
Iter no. 3:
--------------------------------------------
Iter no. 4:
Profesorul Ioana Chiriac nu dorește să predea în intervalul (18, 20)!
--------------------------------------------
Iter no. 5:
Profesorul Mihai Epure nu dorește să predea în intervalul (14, 16)!
--------------------------------------------
Iter no. 6:
--------------------------------------------
Iter no. 7:
Profesorul Cristian Andronescu nu dorește să predea în intervalul (14, 16)!
Profesorul Mihai Epure nu dorește să predea în intervalul (14, 16)!
Profesorul Ruxandra Andronescu nu dorește să predea în intervalul (18, 20)!
--------------------------------------------
Iter no. 8:
Profesorul Ioana Chiriac nu dorește să predea în intervalul (14, 16)!
--------------------------------------------
Iter no. 9:
Profesorul Madalina Popa nu dorește să predea în intervalul

In [92]:
%time run_custom(f'inputs/orar_bonus_exact.yaml', budget=10, ratio=2, runs=10)

--------------------------------------------
Iter no. 1:
Profesorul Alexandru Ilie are pauza prea mare de la 16 pana la 18!
Profesorul Alexandru Ilie are pauza prea mare de la 16 pana la 18!
Profesorul Cristian Adamescu are pauza prea mare de la 10 pana la 16!
Profesorul Pavel Georgescu nu dorește să predea în intervalul (14, 16)!
Profesorul Pavel Georgescu are pauza prea mare de la 12 pana la 14!
Profesorul Roxana Epure are pauza prea mare de la 14 pana la 18!
Profesorul Roxana Popescu are pauza prea mare de la 10 pana la 18!
Number of iterations: 104
Number of states taken into consideration as simulation options: 803,280
Number of states actually generated: 53,552
--------------------------------------------
Iter no. 2:
Profesorul Alexandru Ilie are pauza prea mare de la 14 pana la 18!
Profesorul Cristian Adamescu are pauza prea mare de la 12 pana la 18!
Profesorul Mihai Moldovan are pauza prea mare de la 10 pana la 12!
Profesorul Roxana Popescu are pauza prea mare de la 10 pana la 

In [94]:
%time run_custom(f'inputs/orar_bonus_exact.yaml', budget=30, ratio=2, runs=1)

--------------------------------------------
Iter no. 1:
Profesorul Alexandru Ilie are pauza prea mare de la 14 pana la 18!
Profesorul Cristian Adamescu are pauza prea mare de la 10 pana la 16!
Profesorul Mihai Moldovan are pauza prea mare de la 10 pana la 12!
Profesorul Roxana Popescu are pauza prea mare de la 10 pana la 14!
Number of iterations: 104
Number of states taken into consideration as simulation options: 2,409,735
Number of states actually generated: 160,649
CPU times: total: 19min 50s
Wall time: 19min 51s
